In [ ]:
from manim import *
import numpy as np

In [ ]:
radii = [1, 1, 3, 2, 0]
offsets = [2, 3, 7, 10, 13]

color1 = BLUE
color2 = YELLOW


def find_stable_payments(radii, offsets):
    """
    it finds a stable amount of payments for each person
    """
    h = 1

    change_size = float("inf")
    while change_size > 0.000001:
        sides = []
        areas = []
        for radius, offset in zip(radii, offsets):
            area = PI * radius ** 2 * (np.arctan(h / offset) / TAU)
            side = np.sqrt(area)
            sides.append(side)
            areas.append(area)
        new_h = np.sum(sides)
        
        change_size = np.abs(h - new_h)
        h = new_h

    return h, sides, areas

In [ ]:
def find_leverage_on_the_margin(radii, offsets):
    """
    finds the leverage the last person gets, by increasing their contribution with just a tiny amount
    """
    epsilon = 0.00001
    
    if radii[-1] == 0 and sum(radii) != 0:
        return float("inf")
        
    
    _, _, areas = find_stable_payments(radii, offsets)
    radii = radii.copy()
    radii[-1] += epsilon
    _, _, new_areas = find_stable_payments(radii, offsets)
    total_pot_increase = sum(new_areas) - sum(areas)
    last_contribution_increase = new_areas[-1] - areas[-1]
    leverage = total_pot_increase / last_contribution_increase
    
    return leverage
    

#     # find leverages for each person
#     leverages = []
#     for index in range(len(funcs)):
#         new_funcs = funcs.copy()
#         # modify the fixed value and see how it affects the pot
#         new_funcs[index] = lambda h: payouts[index] + epsilon
#         new_payouts = find_stable_payments(new_funcs)
        
#         pot_difference = (sum(new_payouts) - sum(payouts))
#         leverage = pot_difference / epsilon
#         leverages.append(leverage)
#     return np.array(leverages)

In [ ]:
# i = find_leverage_on_the_margin([1,1,3,2,0], offsets)

In [ ]:
%%manim -vWARNING --format=mp4 -r 1280,720 --fps 60 -o leverage.mp4 Leverage
# %%manim -vWARNING --format=gif -r 1280,720 --fps 60 -o leverage.gif Leverage


class Leverage(Scene):
    
    def updater(self, df):
        # first set of params
        _radii = [tracker.get_value() for tracker in self.radii]
        _offsets = [tracker.get_value() for tracker in self.offsets]
        h, sides, _ = find_stable_payments(_radii, _offsets)
                
        n = len(_radii)
        for i, radius, offset in zip(range(n), _radii, _offsets):
            self.lines[i].become(Line( 
                start=self.ax.c2p(-offset,0),
                end=self.ax.c2p(0, h),
                color=color1,
                stroke_opacity=0.3,
            ))
            self.sectors[i].become(Sector(
                outer_radius=radius * self.axis_scale, 
                color=color1,
                start_angle=np.arctan(h / offset),
                angle=-np.arctan(h / offset),
                arc_center=self.ax.c2p(-offset, 0),
            ))
            self.arcs[i].become(Arc(
                radius=radius * self.axis_scale, 
                arc_center=self.ax.c2p(-offset,0),
                color=BLUE,
                stroke_opacity=0.3,
            ))
            self.arc_closing_lines[i].become(Line(
                start=self.ax.c2p(-offset,radius),
                end=self.ax.c2p(-offset,0),
                color=BLUE,
                stroke_opacity=0.3,
            ))
            
        # second set of params
        _radii2 = [tracker.get_value() for tracker in self.radii2]
        _offsets2 = [tracker.get_value() for tracker in self.offsets2]
        h2, sides2, _ = find_stable_payments(_radii2, _offsets2)
                
        n = len(_radii2)
        for i, radius, offset in zip(range(n), _radii2, _offsets2):
            self.lines2[i].become(Line( 
                start=self.ax.c2p(-offset,0),
                end=self.ax.c2p(0, h2),
                color=color2,
            ))
            self.sectors2[i].become(Sector(
                outer_radius=radius * self.axis_scale, 
                color=color2,
                start_angle=np.arctan(h2 / offset),
                angle=-np.arctan(h2 / offset),
                arc_center=self.ax.c2p(-offset, 0),
            ))
            self.arcs2[i].become(Arc(
                radius=radius * self.axis_scale, 
                arc_center=self.ax.c2p(-offset,0),
                color=BLUE,
            ))
            self.arc_closing_lines2[i].become(Line(
                start=self.ax.c2p(-offset,radius),
                end=self.ax.c2p(-offset,0),
                color=BLUE,
            ))
        
        # set square sizes and positions
        h_acc = 0
        for square, square2, side, side2 in zip(self.squares, self.squares2, sides, sides2):
            square.become(Square(
                side_length=side * self.axis_scale,
                fill_color=color1,
                fill_opacity=1,
                stroke_opacity=0,
                # stroke_color=BLUE,
            ))
            square2.become(Square(
                side_length=side2 * self.axis_scale,
                fill_color=color2,
                fill_opacity=1,
                stroke_opacity=0,
                # stroke_color=BLUE,
            ))
            x, y, _ = self.ax.coords_to_point(side / 2, h_acc + side / 2)
            square.set_x(x)
            square.set_y(y)
            x, y, _ = self.ax.coords_to_point(side2 / 2, h_acc + side2 / 2)
            square2.set_x(x)
            square2.set_y(y)
            h_acc += max(side, side2)
        
        leverage = find_leverage_on_the_margin(_radii2, _offsets2)
        if leverage == float("inf"):
            text = "Leverage:    ∞"
        else:
            leverage = np.clip(leverage, 0, 99)
            text = f"Leverage: {leverage:4.1f}"
        self.leverage_info.become(Text(
            text, font="Monospace"
        ).shift(2 * UL))
            
    
    def construct(self):
        # Creating shapes
        x_range = 15
        y_range = 10
        self.axis_scale = 0.7
        self.ax = Axes(
            x_range=[-x_range, 0],
            y_range=[0, y_range],
            x_length=x_range * self.axis_scale,
            y_length=y_range * self.axis_scale,
            axis_config={"color": BLUE},
            tips=False,
        )
        
        self.radii = [ValueTracker(value) for value in radii]
        self.offsets = [ValueTracker(value) for value in offsets]
        self.radii2 = [ValueTracker(value) for value in radii]
        self.offsets2 = [ValueTracker(value) for value in offsets]
        
        # create objects
        n = len(radii)
        self.arcs = [Arc() for _ in range(n)]
        self.arc_closing_lines = [Line() for _ in range(n)]
        self.lines = [Line() for _ in range(n)]
        self.sectors = [Sector() for _ in range(n)]
        self.squares = [Square() for _ in range(n)]
        # second set of objects
        self.arcs2 = [Arc() for _ in range(n)]
        self.arc_closing_lines2 = [Line() for _ in range(n)]
        self.lines2 = [Line() for _ in range(n)]
        self.sectors2 = [Sector() for _ in range(n)]
        self.squares2 = [Square() for _ in range(n)]
        # info
        self.leverage_info = Text("")
        
        # this is needed to trigger redraw on each frame
        dummy = Mobject()
        dummy.add_updater(lambda x: x)
        self.add(dummy)
        
        self.add_updater(self.updater)
        self.update_self(0.01)
        
        #Showing shapes
        self.add(self.ax)
        # second set of objects
        self.add(*self.arcs2)
        self.add(*self.arc_closing_lines2)
        self.add(*self.lines2)
        self.add(*self.sectors2)
        self.add(*self.squares2)
        # first set of objects
        self.add(*self.arcs)
        self.add(*self.arc_closing_lines)
        self.add(*self.lines)
        self.add(*self.sectors)
        self.add(*self.squares)
        # info
        self.add(self.leverage_info)
        
        self.play(self.radii2[-1].animate.set_value(0.5))
        self.wait(2)
        self.play(self.radii2[-1].animate.set_value(1))
        self.wait(2)
        self.play(self.radii2[-1].animate.set_value(2))
        self.wait(2)
        self.play(self.radii2[-1].animate.set_value(3))
        self.wait(2)
        self.play(self.radii2[-1].animate.set_value(0))
        self.wait(2)
        
        